# Implementing the network that recognizes handwritten digits

In this code, the list sizes contains the number of neurons in the respective layers. So, for example, if we want to create a Network object with 2 neurons in the first layer, 3 neurons in the second layer, and 1 neuron in the final layer, we'd do this with the code:

net = Network([2, 3, 1])


In [44]:
import numpy as np

In [55]:
class Network(object):
        def __init__(self, sizes):
            self.num_layers = len(sizes)
            self.sizes = sizes
            self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
            self.weights = [np.random.randn(y, x) 
                            for x, y in zip(sizes[:-1], sizes[1:])]
        def __repr__(self):
            return f'Network(num_layers={self.num_layers}, sizes={self.sizes}, biases={self.biases}, weights={self.weights})'



The ith column in the weight matrix represents the weights leading out of the ith neuron onto the next layer. In our example we have a 2-3-1 network so for the first element we have two colums, in the first column each row represent the weights from the first neuron to every neuron in the second layer.

So in the matrix w wi,j correspond to the weigth from the jith neuron in the previous layer to the ith neuron in the current layer.

In [56]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

# PRECOND : a is a (n,1) numpy array
def feedforward(self, a):
    """Return the output of the network if "a" is input."""
    #  Give a tuple of (biais, weight) for each layer
    for b, w in zip(self.biases, self.weights):
        a = sigmoid(np.dot(w, a)+b)
        return a

Network.feedforward = feedforward

In [58]:
import random


def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The "training_data" is a list of tuples
        "(x, y)" representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If "test_data" is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
    if test_data: n_test = len(test_data)

    n = len(training_data)
    for j in range(epochs):
        random.shuffle(training_data)
        mini_batches = training_data[k:k+mini_batch_size]
        for k in range(0, n, mini_batch_size):
            self.update_mini_batch(mini_batch, eta) # apply one step of gradient descent
        if test_data:
            print("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))
        else:
            print("Epoch {0} complete".format(j))

Network.SGD = SGD

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 13)


* training data is a list of tuples (x,y) where x is the input and y the expected output
* epochs is the number 
* eta represents the learning rate
* test_data is used to evaluate the network periodically during training, and is not used for training itself it slows down the process

1. Shuffle the training data in order to get different batches each epochs
2. For each mini batch we apply a single step of gradient descent

In [ ]:
def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The "mini_batch" is a list of tuples "(x, y)", and "eta"
        is the learning rate."""

        # Initialize
        nabla_b = [np.zeros(b.shape) for b in self.biases] 
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch: 
            delta_nabla_b, delta_nabla_w = self.backprop(x, y) # compute gradient for each sample
            # Update I
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        # Update II
        self.weights = [w-(eta/len(mini_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]
        
Network.update_mini_batch = update_mini_batch

Rappel : 
En considerant x un vecteur de 28*28 éléments dans chaque elements une intensité de gris. y(x) la valeur attendue de x comme un vecteur de type (0,0,0,1,0,0,0,0,0) represente 4 on considere la fonction de cout (MSE) 

  

$$
C(w,b) \equiv \frac{1}{2n} \sum_x \| y(x) - a\|^2
$$
avec a la valeur sortie de l’algorithme pour une entrée x avec des poids w et des biais b.
1. Initialize : $\nabla b \rightarrow 0 \quad \nabla w \rightarrow 0 $
2. Compute Gradient : 
3. Update I : $ \nabla b \rightarrow \nabla b + \delta \nabla b \quad \nabla w \rightarrow \nabla w + \delta \nabla w $
4. Update II : $w \rightarrow w - \frac{\eta}{m} \sum_x \nabla w_x \quad b \rightarrow b - \frac{\eta}{m} \sum_x \nabla b_x$

where $m$ is the size of the mini batch


In [ ]:
def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)
def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

Network.backprop = backprop

Explanation of the backpropagation algorithm : https://www.youtube.com/watch?v=Ilg3gGewQ5U 

In [ ]:
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
net = Network([784, 30, 10])
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

TypeError: Network() takes no arguments